In [1]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.common.exceptions import NoSuchElementException

from tqdm import tqdm

## Grab Links

First, we need a list of tabs. We can use the total hits ranking to get a bunch of links quickly.

In [50]:
URL = "https://www.ultimate-guitar.com/explore?order=hitstotal_desc&page={}&type[]=Chords"
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--window-size=2560,1440")

driver = webdriver.Chrome(options=chrome_options)
driver.get(URL.format(0))

In [11]:
URL = "https://www.ultimate-guitar.com/explore?order=hitstotal_desc&page={}&type[]=Chords"
def get_page_links(page_number):
    driver.get(URL.format(page_number))
    elements = driver.find_elements(By.TAG_NAME, 'article')
    table = elements[1] # this will *probably* work. It's a magic number

    elements = table.find_elements(By.TAG_NAME, 'a')
    links = [e.get_property('href') for e in elements]

    artist_links = list(filter(lambda s: '/artist/' in s, links))
    page_links = list(filter(lambda s: '/tab/' in s, links))

    return artist_links, page_links

In [13]:
get_page_links(0)

(['https://www.ultimate-guitar.com/artist/ed_sheeran_30232',
  'https://www.ultimate-guitar.com/artist/jeff_buckley_9898',
  'https://www.ultimate-guitar.com/artist/elvis_presley_11125',
  'https://www.ultimate-guitar.com/artist/passenger_21762',
  'https://www.ultimate-guitar.com/artist/john_legend_11714',
  'https://www.ultimate-guitar.com/artist/jason_mraz_10984',
  'https://www.ultimate-guitar.com/artist/radiohead_578',
  'https://www.ultimate-guitar.com/artist/ed_sheeran_30232',
  'https://www.ultimate-guitar.com/artist/vance_joy_37940',
  'https://www.ultimate-guitar.com/artist/bruno_mars_28280',
  'https://www.ultimate-guitar.com/artist/ed_sheeran_30232',
  'https://www.ultimate-guitar.com/artist/oasis_734',
  'https://www.ultimate-guitar.com/artist/pink_floyd_684',
  'https://www.ultimate-guitar.com/artist/justin_bieber_25667',
  'https://www.ultimate-guitar.com/artist/goo_goo_dolls_1375',
  'https://www.ultimate-guitar.com/artist/eagles_1509',
  'https://www.ultimate-guitar.co

In [14]:
all_artist_links = []
all_tab_links = []
for i in tqdm(range(0,101)):
    artist_links, page_links = get_page_links(i)

    all_artist_links += artist_links
    all_tab_links += page_links

  0%|          | 0/101 [00:00<?, ?it/s]

100%|██████████| 101/101 [04:19<00:00,  2.57s/it]


I'll save this data, because I'm getting some refused connections from UG, which means they could be blocking my parser.

In [45]:
import json

with open('tabs.json','w') as f:
    json.dump(all_tab_links, f)

## Chord Extraction

Now that we have page links, we can check those pages and extract the data.

In [101]:
driver.get(all_tab_links[1000])
print(driver.current_url)
driver.find_element(By.XPATH, "//tr/th[contains(text(),'Key')]/../td").text

https://tabs.ultimate-guitar.com/tab/lana-del-rey/born-to-die-chords-1117902


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//tr/th[contains(text(),'Key')]/../td"}
  (Session info: chrome=135.0.7049.42); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
#0 0x5583a1838e6a <unknown>
#1 0x5583a12ea640 <unknown>
#2 0x5583a133bb5a <unknown>
#3 0x5583a133bd51 <unknown>
#4 0x5583a138a344 <unknown>
#5 0x5583a136177d <unknown>
#6 0x5583a13877b5 <unknown>
#7 0x5583a1361523 <unknown>
#8 0x5583a132def3 <unknown>
#9 0x5583a132eb51 <unknown>
#10 0x5583a17fdd2b <unknown>
#11 0x5583a1801c01 <unknown>
#12 0x5583a17e5512 <unknown>
#13 0x5583a1802774 <unknown>
#14 0x5583a17c990f <unknown>
#15 0x5583a1826fb8 <unknown>
#16 0x5583a1827196 <unknown>
#17 0x5583a1837cb6 <unknown>
#18 0x77397649caa4 <unknown>
#19 0x773976529c3c <unknown>


In [ ]:

def get_song_data(URL):
    driver.get(URL)
    #ActionChains(driver).scroll_by_amount(0,10000).perform()

    elements = driver.find_elements(By.CSS_SELECTOR, 'code pre span')
    try:
        key = driver.find_element(By.XPATH, "//tr/th[contains(text(),'Key')]/../td").text
    except NoSuchElementException:
        key = '?'

    data = {
        'link': URL,
        'artist': None,
        'title': None,
        'key': key,
        'text_chords': [e.text for e in elements],
        'prop_chords': [e.get_attribute('data-name') for e in elements]
    }

    return data

URL = all_tab_links[499]
print(get_song_data(URL))

{'link': 'https://tabs.ultimate-guitar.com/tab/i-belong-to-the-zoo/sana-chords-2405013', 'artist': None, 'title': None, 'key': 'C', 'text_chords': ['C', 'G', 'Em', 'D', 'C', 'G', 'Em', 'D', 'C', 'G', 'D', 'G', 'D', 'Em', 'D', 'G', 'D', 'Em', 'D', 'G', 'C', 'G', 'Em', 'G', 'C', 'G', 'D', 'C', 'G', 'Em', 'G', 'C', 'G'], 'prop_chords': ['C', 'G', 'Em', 'D', 'C', 'G', 'Em', 'D', 'C', 'G', 'D', 'G', 'D', 'Em', 'D', 'G', 'D', 'Em', 'D', 'G', 'C', 'G', 'Em', 'G', 'C', 'G', 'D', 'C', 'G', 'Em', 'G', 'C', 'G']}


In [ ]:
all_data = []
chunk = []
n = 0
for tab_link in tqdm(all_tab_links):
    chunk.append(get_song_data(tab_link))


    if len(chunk) > 1000:
        n += 1
        all_data += chunk
        with open(f'data/chunk{n}.json', 'w') as f:
            json.dump(chunk, f)
        
        chunk = []

        print(f'Finished chunk {n}')

print(len(all_data))   

In [ ]:
driver.get()
ActionChains(driver).scroll_by_amount(0,1000).perform()

elements = driver.find_elements(By.CSS_SELECTOR, 'code pre span')
[e.text for e in elements]

['C',
 'G',
 'D',
 'G',
 'D',
 'Em',
 'D',
 'G',
 'D',
 'Em',
 'D',
 'G',
 'C',
 'G',
 'Em',
 'G',
 'C',
 'G',
 'D',
 'C',
 'G',
 'Em',
 'G',
 'C',
 'G',
 'C',
 'C',
 'G',
 'Em',
 'D',
 'C',
 'G',
 'Em',
 'D',
 'C',
 'G',
 'D',
 'G',
 'D',
 'Em',
 'D',
 'G',
 'D',
 'Em',
 'D',
 'G',
 'C',
 'G',
 'Em',
 'G',
 'C',
 'G',
 'D',
 'C',
 'G',
 'Em',
 'G',
 'C',
 'G',
 'D',
 'C',
 'G',
 'Em',
 'G',
 'C',
 'G']